In [ ]:
# ==============================================================================
# CELL 1: IMPORTS AND STYLE CONFIGURATION
# ==============================================================================
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
import os
from tqdm import tqdm  # Use tqdm.notebook for a nice progress bar in Jupyter

# Install scienceplots if you haven't already
# !pip install scienceplots
import scienceplots

# Apply a professional style suitable for IEEE publications
plt.style.use(['science', 'ieee', 'no-latex'])

# This magic command ensures plots are displayed directly in the notebook
%matplotlib inline

print("Libraries imported and style set.")

In [ ]:
# ==============================================================================
# CELL 2: CONFIGURATION AND FILE PATHS
# ==============================================================================

# --- General Configuration ---
OUTPUT_FOLDER = '' # Folder to save the final plot
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- Parameters for this specific analysis (EdDSA (+) version) ---
# NOTE: This script is configured for the unprotected (+) version.
# To analyze other versions, you would change these parameters.

# Paths
# !!! IMPORTANT: UPDATE THESE BASE PATHS TO MATCH YOUR SYSTEM !!!
SIM_FIXED_FOLDER = ''
REAL_FIXED_FOLDER = ''

# Trace parameters
N_TRACES_SIM  = 100             # Number of traces to average for the simulated mean trace
N_TRACES_REAL = 10000           # Number of traces to average for the real mean trace
CYCLES = 3145                   # Total number of clock cycles to analyze

# Processing parameters
PP_CC_SIM = 2               # Samples per clock cycle in simulation
PP_CC_REAL = 16             # Samples per clock cycle in real measurement
NOISE_THRESHOLD_MW = 0.6    # Noise floor to remove from real traces (set to 0 to disable)

# Data loading offsets
# The simulated trace has a 1-sample header (the total number of points)
SIM_OFFSET = 1
# The real trace might have a header or trigger delay
REAL_OFFSET = 32 + 2*PP_CC_REAL

# Plotting parameters
CC_INI = 0
CC_END = CYCLES

print("Configuration and paths are set.")

In [ ]:
# ==============================================================================
# CELL 3: HELPER FUNCTIONS
# ==============================================================================

def load_all_traces(folder_path, num_traces, dtype):
    """Loads a set of traces into a 2D numpy array (samples x traces)."""
    try:
        first_trace_path = os.path.join(folder_path, "trace_0.bin")
        num_samples = np.fromfile(first_trace_path, dtype=dtype).shape[0]
    except FileNotFoundError:
        print(f"ERROR: Could not find trace_0.bin in {folder_path}.")
        return None

    # Pre-allocate memory with shape (num_samples, num_traces)
    all_traces = np.zeros((num_samples, num_traces), dtype=dtype)
    
    print(f"Loading {num_traces} traces from {folder_path}...")
    for i in tqdm(range(num_traces)):
        trace_file = os.path.join(folder_path, f"trace_{i}.bin")
        if os.path.exists(trace_file):
            all_traces[:, i] = np.fromfile(trace_file, dtype=dtype)
        else:
            print(f"Warning: Trace file not found and skipped: {trace_file}")
            
    return all_traces

def process_sim_trace(sim_data, offset, pp_cc):
    """Averages, applies offset, and extracts posedge samples from simulated data."""
    # 1. Trim initial offset from the entire dataset
    trimmed_data = sim_data[offset:, :]
    # 2. Calculate the mean trace
    mean_trace = np.mean(trimmed_data, axis=1)
    # 3. Extract only the posedge samples (every second sample)
    return mean_trace[::pp_cc]

def process_real_trace(real_data, offset, total_cycles, pp_cc, noise_threshold):
    """Averages, applies offset, thresholds, and downsamples the real data."""
    # 1. Trim initial offset
    trimmed_data = real_data[offset:, :]
    # 2. Calculate the mean trace
    mean_trace = np.mean(trimmed_data, axis=1)
    # 3. Apply noise threshold to focus on dynamic power
    mean_trace[np.abs(mean_trace) < noise_threshold] = 0
    
    # 4. Downsample to 1 point per cycle
    downsampled = np.zeros(total_cycles)
    if len(mean_trace) < total_cycles * pp_cc:
        print(f"Warning: Real trace length ({len(mean_trace)}) is shorter than expected ({total_cycles * pp_cc}).")
        total_cycles = len(mean_trace) // pp_cc

    for i in range(total_cycles):
        start_idx, end_idx = i * pp_cc, (i + 1) * pp_cc
        cycle_window = mean_trace[start_idx:end_idx]
        if len(cycle_window) > 0:
            max_abs_idx = np.argmax(np.abs(cycle_window))
            downsampled[i] = cycle_window[max_abs_idx]
            
    return downsampled

print("Helper functions defined.")

In [ ]:
# ==============================================================================
# CELL 4: DATA PROCESSING AND CORRELATION
# ==============================================================================

# --- Load and Process SIMULATED trace ---
print("--- Processing Simulated Traces ---")
sim_fixed_data = load_all_traces(SIM_FIXED_FOLDER, N_TRACES_SIM, np.int32)
sim_trace_final = process_sim_trace(sim_fixed_data, SIM_OFFSET, PP_CC_SIM)

# --- Load and Process REAL trace ---
print("\n--- Processing Real Traces ---")
real_fixed_data = load_all_traces(REAL_FIXED_FOLDER, N_TRACES_REAL, np.float32)
real_trace_final = process_real_trace(real_fixed_data, REAL_OFFSET, CYCLES, PP_CC_REAL, NOISE_THRESHOLD_MW)

# --- Final Alignment and Correlation ---
# Ensure both final traces have the same length for comparison
final_len = min(len(sim_trace_final), len(real_trace_final))
sim_trace_final = sim_trace_final[:final_len]
real_trace_final = real_trace_final[:final_len]

correlation_coefficient, _ = ss.pearsonr(sim_trace_final, real_trace_final)
print(f"\nFinal trace length for comparison: {final_len} clock cycles.")
print(f"Pearson Correlation Coefficient: {correlation_coefficient:.4f}")

In [ ]:
# ==============================================================================
# CELL 5: GENERATE AND SAVE THE FINAL PLOT
# ==============================================================================

# --- 1. Normalize Traces for Visual Comparison ---
# Min-Max normalization to scale both traces to the [0, 1] range
sim_min, sim_max = np.min(sim_trace_final), np.max(sim_trace_final)
sim_normalized = (sim_trace_final - sim_min) / (sim_max - sim_min)

real_min, real_max = np.min(real_trace_final), np.max(real_trace_final)
real_normalized = (real_trace_final - real_min) / (real_max - real_min)

# --- 2. Create Figure and Axes ---
fig, ax = plt.subplots(figsize=(12, 4)) # Good size for two-column paper format

# --- 3. Plot the Normalized Traces ---
x_axis = np.arange(len(sim_normalized))

ax.plot(x_axis, real_normalized,
        label='Real Trace (Normalized)',
        color='blue',
        linewidth=0.6)

ax.plot(x_axis, sim_normalized,
        label='Simulated Trace (Normalized)',
        color='green',
        linewidth=0.5,
        alpha=0.7)

# --- 4. Customize Axes, Title, and Legend ---
ax.set_title(f'Comparison of Simulated vs. Real Power Traces\nPearson Correlation: {correlation_coefficient:.4f}', fontsize=16)
ax.set_xlabel('Clock Cycles', fontsize=14, fontweight='bold')
ax.set_ylabel('Normalized Amplitude', fontsize=14, fontweight='bold')

ax.set_xlim(CC_INI, CC_END)
ax.set_ylim(-0.05, 1.1) # Add some padding

# Add a subtle grid for better readability
ax.grid(True, linestyle='--', alpha=0.5, zorder=0)
ax.set_axisbelow(True)

# Customize the legend
legend = ax.legend(loc='best', fontsize=14, frameon=True, fancybox=True)
legend.get_frame().set_alpha(0.9)

# --- 5. Save and Display the Figure ---
output_file_path = os.path.join(OUTPUT_FOLDER, 'power_correlation.pdf')
plt.tight_layout()
plt.savefig(output_file_path, dpi=300)

print(f"\nPlot saved to {output_file_path}")
plt.show()